<a href="https://colab.research.google.com/github/AlineSchaub/DataScience21/blob/7_Block/Kopie_von_Exercise_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment - Image Classification with CNNs
![CIFAR10](https://miro.medium.com/max/964/1*syyml8q8s1Yt-iEea5m1Ag.png)

[CIFAR 10](https://www.cs.toronto.edu/~kriz/cifar.html) is a small toy data set for image classifaction. 

* It has 60k images from ten diffenrent classes (like car, airplane, horse,..)
* The RGB images are of size 32x32 pixels  

In [53]:
#import needed libs
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
import os

#some vars to controll the training -> use them in your code
batch_size = 32
num_classes = 10 #fixed!
epochs = 10

# Get the data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Task:
* visualize some sample images from different classes
* build and train the ***AlexNet*** CNN architecture from the lecture, use the following Keras layers:
    * overview: https://keras.io/api/layers/
    * 2d convolution: https://keras.io/api/layers/convolution_layers/convolution2d/
    * Fully Connected (=Matrix Multiplication MLP): https://keras.io/api/layers/core_layers/dense/
    * Pooling: https://keras.io/api/layers/pooling_layers/max_pooling2d/   
* use 'relu' functions for the non-linear activation -> https://keras.io/api/layers/activation_layers/relu/
* optimize the learning hyper-paramters to get the best  test accuracy

In [55]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras import activations

#RGB (3) und Bildgröße 32 auf 32
inputs = keras.Input(shape=(32, 32, 3))


#Bilder zuschneiden
x_1 = CenterCrop(height=30, width=30)(inputs)


# Rescale images to [0, 1]
x_2 = Rescaling(scale=1.0 / 255)(x_1)

# 2d Convolution layer
x_3 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation="relu")(x_2)

#fully connected layer
x_4 = layers.Dense(32, activation='relu')(x_3)

#Pooling layer
x_5 = layers.MaxPooling2D(pool_size=(3, 3))(x_4)


x_6 = layers.GlobalAveragePooling2D()(x_5)



In [56]:

outputs = layers.Dense(num_classes, activation=activations.relu)(x_6)

In [57]:
model = keras.Model(inputs=inputs, outputs=outputs)

In [58]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
center_crop_4 (CenterCrop)   (None, 30, 30, 3)         0         
_________________________________________________________________
rescaling_4 (Rescaling)      (None, 30, 30, 3)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 32)        896       
_________________________________________________________________
dense_13 (Dense)             (None, 28, 28, 32)        1056      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 9, 9, 32)          0         
_________________________________________________________________
global_average_pooling2d_4 ( (None, 32)                0   

In [59]:
#optimize + loss
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=1e-3),
              loss=keras.losses.CategoricalCrossentropy(),
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")])

In [60]:
#batch size und epochs von oben

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Epoch 1/10


ValueError: ignored

In [42]:
print(history.history)

{'loss': [4.970157623291016, 4.751426696777344, 3.815335988998413, 3.4123265743255615, 3.452756643295288, 3.4460816383361816, 3.442960262298584, 3.407966136932373, nan, nan]}


**Versuch 2:**

In [49]:
#Metrics

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc")],
)




Epoch 1/10


ValueError: ignored